# LIN 373 Machine Learning Toolbox for Text Analysis

# Homework 2 - due Monday Sep 26 2022 at 11:59pm

For this homework you will hand in (upload) to canvas your code named ``hw2_YourEID.ipynb``.

__Before submitting__, please reset your kernel and rerun everything from the beginning (`Kernel` >> `Restart and Run All`) an ensure your code outputs the correct answer.

A perfect solution for this homework is worth **100** points, and there is a bonus problem at the end that is worth **5 points**. For non-programming tasks, you must show work to get partial credit. For programming tasks, make sure that your code can run using Python 3.5+. If you cannot complete a problem, include as much pseudocode as possible for partial credit. However, make sure it does not have any output errors. **If there are any output errors, half of the points for that problem will be automatically deducted.**

Collaboration: you are free to discuss the homework assignments with other students and work towards solutions together.  However, all of the code you write must be your own! There is a channel on Slack where you can look for a study group.

Review extension and academic dishonesty policy here: https://jessyli.com/courses/lin373n

For typing up your answers to problems 1, 2 and 3, information can be found about Markdown cells for Jupyter Notebooks here: https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html


### Please list any collaborators here:
- Ethan Glass


# Problem 1: Naive Bayes

We'll do a bit of manual parameter estimation here.

## **(a)** (8 points) 
Calculate the sufficient parameters for Naive Bayes using the data in the figure below, that
is, the prior class probabilities and the conditional probabilities for all of the symbols.

doc | X | Y
-- | --| --
d1 | a b b b c b | +
d2 | c a c c c b | -
d3 | c c c b | -
d4 | b a b b b | +
d5 | b c a b b | ???


* P(+) = 1/2
* P(-) = 1/2
* P(a | +) = 2/11
* P(a | -) = 1/10
* P(b | +) = 8/11
* P(b | -) = 2/10
* P(c | +) = 1/11
* P(c | -) = 7/10

## **(b)** (7 points) 
Using these, calculate the most likely class (pos or neg) for the unlabeled example (d5, labeled ???).

In [1]:
pos_prob = (1/2 * 2/11 * 1/11 * 8/11 * 8/11 * 8/11)
neg_prob = (1/2 * 1/10 * 2/10 * 2/10 * 2/10 * 7/10)
print('Probability that d5 is positive: {}'.format(pos_prob))
print('Probability that d5 is negative: {}'.format(neg_prob))
print("d5 is more likely to be positive.")

Probability that d5 is positive: 0.0031791171740628748
Probability that d5 is negative: 0.00028
d5 is more likely to be positive.


# Problem 2: Language Modeling

In this problem, we will implement our own Shakespeare and Austen language models!

### Data

We will be working with a few different files under the `data` subdirectory. 
- `shakespeare.txt`
- `shakespeare_short.txt`
- `austen.txt`
- `austen_short.txt`
- `test_short.txt`

<mark>You should use the "short" text files to do testing and debugging</mark> as the other files will take a little while (but should be under 10 mins for a 8G RAM laptop) to run! 

### Note re NLTK

The goal of this problem is to think about how to implement language models from first principles. Thus, **the *ONLY* allowable functions from NTLK in this problem are `sent_tokenize` and `word_tokenize`.**

## (a) Clean text. (5 points) 

Create a function `sent_transform(sent_string)`, such that when given a sentence as a string, it would return a `list` of words, lower-cased. Use NLTK's `word_tokenize` function to tokenize the sentence. For now, you can use any random string to test this function.

```
>>> sent_transform("ROSALIND. Why, whither shall we go?")
['rosalind', '.', 'why', ',', 'whither', 'shall', 'we', 'go', '?']
```

In [2]:
from nltk.tokenize import word_tokenize

def sent_transform(sent_string):
    
    sent_string = sent_string.lower()
    
    return word_tokenize(sent_string)

sent_transform("ROSALIND. Why, whither shall we go?")

['rosalind', '.', 'why', ',', 'whither', 'shall', 'we', 'go', '?']

## (b) Find n-grams. (10 points) 

Create a function `make_ngram_tuples(words, n)` that returns a sequence of all n-grams seen in the input, in order.  The function returns a sequence of tuples where each tuple is of the form `(context, word)`.  The context is a tuple of the preceding n−1 words for each word; if the number of preceding words is smaller than n−1, place a `<s>` symbol in place of each missing context.  Close each sentence with an additional token `</s>`.  You can assumen n>1, that is, we are interested in enumerating bigrams, trigrams etc, and not unigrams.

For now, you can use any random string to test this function.

```
>>> samples = [’she’, ’eats’, ’happily’ ’.’]
>>> make_ngram_tuples(samples, 2)
[((’<s>’,), ’she’), ((’she’,), ’eats’), ((’eats’,), ’happily’), ((’happily’,), ’.’), ((’.’,), ’</s>’)]
>>> make_ngram_tuples(samples, 3)

[((’<s>’, ’<s>’), ’she’), ((’<s>’, ’she’), ’eats’), ((’she’, ’eats’), ’happily’),((’eats’, ’happily’), ’.’), ((’happily’, ’.’), ’</s>’)]
```

In [3]:
def make_ngram_tuples(words, n):
    
    tuples = []
    length = len(words)
    starter = ()
    counter = 1

    # adds n - 1 sentence starters and appends the first tuple to tuples
    for num in range(n - 1): 
        starter += ('<s>',)
    starter = (starter, words[0])
    tuples.append(starter)
        
    # iterates through each tuple in the list
    for item in tuples:
        first_item = () # this will be the first item in new_tuple
    
        # if n > 2, the loop will get all elements (except the first) from the first element, which is a tuple, in item
        for num in range(1, n - 1):
            first_item += (item[0][num],)

        # add the second element in item to first_item
        first_item += (item[1],)
        
        if counter < length:
            # new_tuple will be a nested tuple
            # first element is first_item, a tuple
            # second element is the next word 
            new_tuple = (first_item, words[counter])
            tuples.append(new_tuple)
            counter += 1
        
        # we have reached the last word and need to end it with </s>
        else: 
            new_tuple = (first_item, '</s>')
            tuples.append(new_tuple)
            break
            
    return tuples

print(make_ngram_tuples(['she', 'eats', 'happily', '.'], 2))
print()
print(make_ngram_tuples(['she', 'eats', 'happily', '.'], 3))

[(('<s>',), 'she'), (('she',), 'eats'), (('eats',), 'happily'), (('happily',), '.'), (('.',), '</s>')]

[(('<s>', '<s>'), 'she'), (('<s>', 'she'), 'eats'), (('she', 'eats'), 'happily'), (('eats', 'happily'), '.'), (('happily', '.'), '</s>')]


## (c)  Building an n-gram language model

We are now ready to estimate a bigram language model from the training data.  We will use **Laplace smoothing (add-1)**.

### (c1) Process the training file (8 points)

We will first need to transform a file of plain text into a list of tokenized "sentences", where each "sentence" is a list of *lower-cased* words. Implement a function `process_text(textfile)` to do so. Assume that the path of `textfile` is of a form like `data/austen_short.txt`, i.e., relative paths. Make sure that you call `sent_transform` in this function.

```
>>> processed_sents = process_text("data/austen_short.txt")
>>> print(processed_sents[10])
['he', 'considered', 'the', 'blessing', 'of', 'beauty', 'as', 'inferior', 'only', 'to', 'the', 'blessing', 'of', 'a', 'baronetcy', ';', 'and', 'the', 'sir', 'walter', 'elliot', ',', 'who', 'united', 'these', 'gifts', ',', 'was', 'the', 'constant', 'object', 'of', 'his', 'warmest', 'respect', 'and', 'devotion', '.']
```

In [4]:
from nltk.tokenize import sent_tokenize

def process_text(textfile):
    
    sentences = []
    
    with open(textfile, encoding='utf-8' ) as f:
        content = f.read() 
        tokenized_sentences = sent_tokenize(content)
        for sentence in tokenized_sentences:
            transformed_sentences = sent_transform(sentence)
            sentences.append(transformed_sentences)
  
    return sentences

processed_sents = process_text(r"C:\Users\veron\OneDrive\Documents\LIN 373N\HW2\data\data\austen_short.txt")
print(processed_sents[10])

['he', 'considered', 'the', 'blessing', 'of', 'beauty', 'as', 'inferior', 'only', 'to', 'the', 'blessing', 'of', 'a', 'baronetcy', ';', 'and', 'the', 'sir', 'walter', 'elliot', ',', 'who', 'united', 'these', 'gifts', ',', 'was', 'the', 'constant', 'object', 'of', 'his', 'warmest', 'respect', 'and', 'devotion', '.']


### (c2) Vocabulary (8 points).

The language model ought to be able to handle words not seen in the training data.  Such words will most certainly appear if the LM is used in some application to estimate the likelihood of new data.   There  are  many  ways  to  incorporate  unknown  vocabulary  in  a  language  model.   In  this problem, we will take all words that appear only once and replace them with the symbol `<UNK>`. Thus, at test time, if we encounter an out-of-vocabulary word, we can resort to replacing the word with `<UNK>` which has well-formed probabilities.

First, implement a function `get_vocab(tokenized_sents)` where the parameter `tokenized_sents` is a list of tokenized "sentences" (where each "sentence" is a list of lower-cased words) returned by the function `process_text`. The function `get_vocab` will return a `set` of all unique words in `tokenized_sents` that appeared more than once.
Note: do not forget to add three extra tokens to the vocabulary: `<s>`, `</s>`, and `<UNK>`.



```
>>> vocab = get_vocab(processed_sents)
>>> print(len(vocab))
302
```

In [5]:
from collections import Counter
def get_vocab(tokenized_sents):
    
    multiple_words = set() # create an empty set
    all_words = [word for sentence in tokenized_sents for word in sentence] # combine all sentences into one list
    
    counts = Counter(all_words) # use counter to get counts of words
    multiples = [key for key, value in counts.items() if value > 1] # make a list 'multiples' of words w/ more than 1 count
    
    multiple_words.update(multiples) # add multiples
    multiple_words.update(['<s>', '</s>', '<UNK>']) # add the three extra tokens
    
    return multiple_words

vocab = get_vocab(processed_sents)
print(len(vocab))

302


### (c3) Process unknown words (8 points).

Write a function `process_unk(tokenized_sents, vocab)` that takes in (1) tokenized sentences returned by `process_text`, and (2) a vocabulary (set of words) returned by `get_vocab`. The function returns a list of tokenized sentences that is the same as `tokenized_sents` except that all words appearing only once will be replaced by the symbol `<UNK>`.

```
>>> print(processed_sents[3])
['of', 'south', 'park', ',', 'in', 'the', 'county', 'of', 'gloucester', ',', 'by', 'which', 'lady', '(', 'who', 'died', '1800', ')', 'he', 'has', 'issue', 'elizabeth', ',', 'born', 'june', '1', ',', '1785', ';', 'anne', ',', 'born', 'august', '9', ',', '1787', ';', 'a', 'still-born', 'son', ',', 'november', '5', ',', '1789', ';', 'mary', ',', 'born', 'november', '20', ',', '1791', '.', "''"]
>>> processed_unk = process_unk(processed_sents, vocab)
>>> print(processed_unk[3])
['of', '<UNK>', '<UNK>', ',', 'in', 'the', 'county', 'of', '<UNK>', ',', 'by', 'which', 'lady', '(', 'who', 'died', '<UNK>', ')', 'he', 'has', '<UNK>', 'elizabeth', ',', 'born', '<UNK>', '1', ',', '<UNK>', ';', 'anne', ',', 'born', '<UNK>', '<UNK>', ',', '<UNK>', ';', 'a', '<UNK>', 'son', ',', 'november', '<UNK>', ',', '<UNK>', ';', 'mary', ',', 'born', 'november', '<UNK>', ',', '<UNK>', '.', "''"]
```

In [6]:
def process_unk(tokenized_sents, vocab):
    
    new_tokenized_sents = []
    
    for sentence in tokenized_sents:
        
        new_sentence = []
        
        for word in sentence:
            
            if word in vocab:
                new_sentence.append(word)
            else:
                new_sentence.append('<UNK>')
        
        new_tokenized_sents.append(new_sentence)
    
    return new_tokenized_sents

print(processed_sents[3])
processed_unk = process_unk(processed_sents, vocab)
print(processed_unk[3])

['of', 'south', 'park', ',', 'in', 'the', 'county', 'of', 'gloucester', ',', 'by', 'which', 'lady', '(', 'who', 'died', '1800', ')', 'he', 'has', 'issue', 'elizabeth', ',', 'born', 'june', '1', ',', '1785', ';', 'anne', ',', 'born', 'august', '9', ',', '1787', ';', 'a', 'still-born', 'son', ',', 'november', '5', ',', '1789', ';', 'mary', ',', 'born', 'november', '20', ',', '1791', '.', "''"]
['of', '<UNK>', '<UNK>', ',', 'in', 'the', 'county', 'of', '<UNK>', ',', 'by', 'which', 'lady', '(', 'who', 'died', '<UNK>', ')', 'he', 'has', '<UNK>', 'elizabeth', ',', 'born', '<UNK>', '1', ',', '<UNK>', ';', 'anne', ',', 'born', '<UNK>', '<UNK>', ',', '<UNK>', ';', 'a', '<UNK>', 'son', ',', 'november', '<UNK>', ',', '<UNK>', ';', 'mary', ',', 'born', 'november', '<UNK>', ',', '<UNK>', '.', "''"]


### (c4) N-gram frequencies (10 points).

We now need to get the frequency counts -- which will allow us to calculate the conditional frequency distribution for our language model. Write a function `get_freq_dist(tokenized_sents, n)` that takes in (1) a list of tokenized sentences (such as one returned by `process_unk`), and (2) the number `n` that denotes the order of the n-gram (`n=2` means bigram, `n=3` means trigram, etc). The function will return a dictionary `freqdict` (`freqdict` can be a `Counter`) such that `freqdict[context][word]` records the number of times `word` follows `context`. You can think of `context` as the first element of a tuple in a list returned by `make_ngram_tuples`, and `word` as the second element of that tuple.

```
>>> freqdict = get_freq_dict(processed_unk, 2)
>>> print(freqdict[('of',)])
Counter({'<UNK>': 25, 'the': 17, 'his': 6, 'her': 6, 'kellynch': 4, 'a': 4, 'being': 4, 'their': 3, 'it': 3, 'charles': 2, 'somerset': 2, 'any': 2, 'very': 2, 'mind': 2, 'life': 2, 'all': 2, 'mr': 2, 'them': 2, 'himself': 1, 'mary': 1, 'high': 1, 'baronet': 1, 'sir': 1, 'character': 1, 'real': 1, 'ever': 1, 'respectability': 1, '.': 1, 'youth': 1, 'and': 1, 'elliot': 1, 'inferior': 1, 'again': 1, 'economy': 1, 'which': 1, ';': 1})
```

In [7]:
from collections import defaultdict

def get_freq_dict(tokenized_sents, n):
    
    freqdict = defaultdict(Counter)
    all_ngram = []
    
    # make ngrams of each sentence, then combine them all into one list 
    for sentence in tokenized_sents:
        all_ngram += make_ngram_tuples(sentence, n)
    
    for context, word in all_ngram:
        freqdict[context][word] += 1
        
    return freqdict
 
freqdict = get_freq_dict(processed_unk, 2)
print(freqdict[('of',)])

Counter({'<UNK>': 25, 'the': 17, 'his': 6, 'her': 6, 'kellynch': 4, 'a': 4, 'being': 4, 'their': 3, 'it': 3, 'charles': 2, 'somerset': 2, 'any': 2, 'very': 2, 'mind': 2, 'life': 2, 'all': 2, 'mr': 2, 'them': 2, 'himself': 1, 'mary': 1, 'high': 1, 'baronet': 1, 'sir': 1, 'character': 1, 'real': 1, 'ever': 1, 'respectability': 1, '.': 1, 'youth': 1, 'and': 1, 'elliot': 1, 'inferior': 1, 'again': 1, 'economy': 1, 'which': 1, ';': 1})


### (c5) The langauge model

We are now ready to put everything together and make our langauge model! Below we have written the function `build_ngram_model(textfile, n)` that takes in a text file, the value `n` that signals the order of our n-gram, and returns a language model in the form of a `namedtuple`. All we need to do is to call the various functions you've implemented so far in (c)! (**No implementation required here**).

A `namedtuple` is a versatile data structure that allows one to associate multiple data fields with one variable. Below, we created a `namedtuple` called `LanguageModel`; this `LanguageModel` consists of the following information: the n-gram order `n`, the frequency distribution dictionary `fd`, and the vocabulary (as a `set` of words) `vocab`. Now, after we make a `LanguageModel`, we will be able to access these fields using the `dot` operator, for example, `toy_lm.vocab`.

In [8]:
## DO NOT MODIFY THIS CELL, but you need to run it.

from collections import namedtuple

def build_ngram_model(textfile, n):
    LanguageModel = namedtuple('LanguageModel', ['n', 'fd', 'vocab'])
    psents = process_text(textfile)
    vocab = get_vocab(psents)
    psentsunk = process_unk(psents, vocab)
    fd = get_freq_dict(psentsunk, n)
    return LanguageModel(n, fd, vocab)

toy_lm = build_ngram_model(r"C:\Users\veron\OneDrive\Documents\LIN 373N\HW2\data\data\austen_short.txt", 2)


### (c6) Log conditional probabilities. (10 points)

The heart of the language model above is just the frequency dictionary. To make our model functional, we will need to use the frequency dictionary to calculate (log) conditional probabilities. Write a function `log_prob(lm, context, word)` that takes in a language model `lm`, `context` in the form of a tuple, and a `word`. The function returns the *add-1 (Laplacian) smoothed* conditional probability values `log P(word|context)`.

We would like to calculate log probabilities, instead of raw probability values, because ultipmately we will use the language model to calculate the likelihood of a text. Multiplying many very small raw probability values may result in underflow issues (and inaccuracies) in any programming language.

You will need to get the size of the vocabulary when writing this function. Keep in mind that the `<UNK>`, `<s>`, and `</s>` symbols should all be a part of your vocabulary.

*Please use log with base 2 for this problem!*


```
>>> log_prob(toy_lm, ('.',), '</s>')
-2.451695969857692
```

In [9]:
import math

def log_prob(lm, context, word):
    
    n = lm[0]
    fd = lm[1]
    vocab = lm[2]
    size_vocab = len(lm[2])
    
    # count of word following context, + 1
    numerator = fd[context][word] + 1 
    # count of all context occurrences 
    denom = sum(fd[context].values()) + size_vocab
    
    log_prob = math.log(numerator / denom, 2)
    
    return log_prob
    

print(log_prob(toy_lm, ('.',), '</s>'))

-2.451695969857692


### (c7) Perplexity (16 points)

Our final step to make our langauge model functional would be to calculate perplexity of a document (e.g., a new text file). Implement the function `get_ppl(lm, newfile)` that returns the perplexity of `newfile` given language model `lm` that you built using `build_ngram_model`.

Be mindful to check whether a word appears in the language model's vocabulary; if not, replace with `<UNK>`.

```
>>> get_ppl(toy_lm, "data/test_short.txt")
>>> 51.98841239479734
```

In [10]:
def get_ppl(lm, testfile):
    
    n = lm[0]
    fd = lm[1]
    vocab = lm[2]
    
    test_size = 0
    test_ngram = []
    sum_log_prob = 0
    
    # process text 
    test = process_text(testfile)
    
    # replace words in testfile with <UNK> 
    process_test = process_unk(test, vocab)
            
    # add sentence starter and ender to each length of sentence
    # also create ngrams for each sentence
    for sentence in test:
        test_size += len(sentence) + 2 # length of sentence plus sentence start and end
    
    # create freqdict 
    test_freq = get_freq_dict(process_test, n)
    
    for context_key in test_freq:
        for word_key in test_freq[context_key]:
            count = test_freq[context_key][word_key]
            prob = log_prob(lm, context_key, word_key) 
            sum_log_prob += (prob * count)
    
    return 2 ** ((sum_log_prob*(-1)) / test_size)
    
    

get_ppl(toy_lm, r"C:\Users\veron\OneDrive\Documents\LIN 373N\HW2\data\data\test_short.txt")

51.988412394797635

## (d) Authorship attribution (10 points)

If we have built language models of multiple authors, they can be used to check the author of an unknown piece of writing. Concretely, for the unknown text, we can run the file through each known author's language model, and use perplexity to predict which author the unknown text is most likely to belong to.

In this problem, build two **bigram** models:
- a  Shakespeare language model
- a  Jane Austen language model

Make sure to train these models from the full text. Once you have trained both models, use the `get_ppl` function from each language model on the file `test.txt`. 

Who wrote the text?

In [11]:
shakespeare = build_ngram_model(r"C:\Users\veron\OneDrive\Documents\LIN 373N\HW2\data\data\shakespeare.txt", 2)
austen = build_ngram_model(r"C:\Users\veron\OneDrive\Documents\LIN 373N\HW2\data\data\austen.txt", 2)
test = r"C:\Users\veron\OneDrive\Documents\LIN 373N\HW2\data\data\test.txt"

ppl_shakespeare = get_ppl(shakespeare, test)
ppl_austen = get_ppl(austen, test)
print(ppl_shakespeare)
print(ppl_austen)

511.31765795186305
385.1354241879401


A lower perplexity indicates a better model, so it is more likely that Jane Austen wrote the test text. 

## (e) Bonus: Random text generator (5 points)

Now, we are ready to generate some Shakespeare text! Starting with the sentence start symbol `<s>`, at each step, use the previously generated word as context, and sample one of the top 5 words that follow this word. We stop whenever we hit `</s>`, or when we have generated a 20-word sentence, whichever is earlier.

Implement a function `text_generator(bigramlm, k)` that takes a bigram langauge model---trained on our Shakespeare text---as input, and generates `k` random sentences.